In [67]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pybaseball import lahman
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
years = np.array(["2015","2016","2017","2018","2019","2020"])
teams = np.array(["ATL", "BOS", "BRK", "CHI", "CHO", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"])

In [94]:
def HtWght(team, year):
    html = urlopen('https://www.basketball-reference.com/teams/'+ team + '/'+ year +'.html')
    bs = BeautifulSoup(html.read())
    tables = bs.find_all('table')
    stats = pd.read_html(str(tables))
    Bio = pd.DataFrame(stats[0][:-1])
    Bio["Year"] = year
    return(Bio)

Bio = pd.DataFrame()
for y in years:
    for i in teams:
        t = HtWght(i, y)
        Bio = pd.concat([Bio, t])
Bio = Bio.reset_index()
for r in range(len(Bio)):
    sp = Bio["Ht"].str.split("-")
    feet = sp[r][0]
    inch = sp[r][1]
    Bio["Ht"][r] = int(feet)*12+int(inch)

In [10]:
def PerGame(team, year):
    html = urlopen('https://www.basketball-reference.com/teams/'+ team + '/'+ year +'.html')
    bs = BeautifulSoup(html.read())
    tables = bs.find_all('table')
    stats = pd.read_html(str(tables))
    Per = pd.DataFrame(stats[1][:-1])
    Per["Year"] = year
    return(Per)

Per = pd.DataFrame()
for y in years:
    for i in teams:
        t = PerGame(i, y)
        Per = pd.concat([Per, t])

In [11]:
def Advanced(team, year):
    html = urlopen('https://www.basketball-reference.com/teams/'+ team + '/'+ year +'.html')
    bs = BeautifulSoup(html.read())
    tables = bs.find_all('table')
    stats = pd.read_html(str(tables))
    Adv = pd.DataFrame(stats[2][:-1])
    Adv["Year"] = year
    return(Adv)

Adv = pd.DataFrame()
for y in years:
    for i in teams:
        t = Advanced(i, y)
        Adv = pd.concat([Adv, t])

In [98]:
Per = Per.rename(columns = {"Unnamed: 1":"Player Name"})
Adv = Adv.rename(columns = {"Unnamed: 1":"Player Name"})
Bio = Bio.rename(columns = {"Player":"Player Name"})
NBA = pd.merge(Per, Adv, on = ['Player Name', 'Year', 'Age', 'G'], how = "inner")
NBA = pd.merge(NBA, Bio, on = ['Player Name', 'Year'], how = "inner")
del NBA['Unnamed: 6']
del NBA['Unnamed: 17']
del NBA['Unnamed: 22']
del NBA["index"]
del NBA["No."]
del NBA['Rk_x']
del NBA['Rk_y']
del NBA['MP_y']
NBA = NBA.rename(columns = {"MP_x":"MP"})
NBA

,Player Name,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,OBPM,DBPM,BPM,VORP,Pos,Ht,Wt,Birth Date,Exp,College
0,Paul Millsap,29,73,73,32.7,6.1,12.7,0.476,1.1,3.0,...,1.7,1.7,3.4,3.3,PF,79,257,"February 10, 1985",8,Louisiana Tech
1,Kyle Korver,33,75,75,32.2,3.9,8.0,0.487,2.9,6.0,...,2.4,0.4,2.8,3.0,SG,79,212,"March 17, 1981",11,Creighton
2,DeMarre Carroll,28,70,69,31.3,4.5,9.3,0.487,1.7,4.3,...,1.2,0.4,1.7,2.0,SF,78,215,"July 27, 1986",5,"Vanderbilt, Missouri"
3,Al Horford,28,76,76,30.5,6.8,12.7,0.538,0.1,0.5,...,2.6,1.5,4.1,3.6,C,81,240,"June 3, 1986",7,Florida
4,Dennis Schröder,21,77,10,19.7,3.7,8.6,0.427,0.7,1.9,...,-0.1,-1.2,-1.3,0.2,PG,75,172,"September 15, 1993",1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3815,C.J. Miles,32,10,0,16.1,1.9,5.9,0.322,1.1,3.5,...,-1.8,0.1,-1.8,0.0,SF,78,220,"March 18, 1987",14,NaN
3816,Gary Payton,27,29,17,14.9,1.7,4.0,0.414,0.4,1.6,...,-3.2,1.6,-1.6,0.0,SF,75,190,"December 1, 1992",3,Oregon State
3817,Jerian Grant,27,6,0,13.3,1.7,4.5,0.370,0.3,1.3,...,-4.8,-2.0,-6.8,-0.1,SG,76,205,"October 9, 1992",4,Notre Dame
3818,Garrison Mathews,23,18,0,12.6,1.3,3.1,0.429,1.1,2.6,...,-0.6,-0.5,-1.1,0.1,SG,77,215,"October 24, 1996",R,Lipscomb


In [99]:
NBA.to_csv("Merge.csv")